# Nutrition Image Analysis using CNN

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [2]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [3]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'C:\Users\Varun\Downloads\Nutrition Analysis Using Image Classification-20211029T043804Z-001\Nutrition Analysis Using Image Classification\Dataset\TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'C:\Users\Varun\Downloads\Nutrition Analysis Using Image Classification-20211029T043804Z-001\Nutrition Analysis Using Image Classification\Dataset\TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [4]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [5]:
print(x_test.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [6]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

### Creating the model

In [7]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [8]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

### Compiling the model

In [9]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Fitting the model

In [10]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\Varun\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
526/526 [==============================] - 49s 92ms/step - loss: 0.1623 - accuracy: 0.9398 - val_loss: 0.1247 - val_accuracy: 0.9450
Epoch 2/20
526/526 [==============================] - 15s 28ms/step - loss: 0.0232 - accuracy: 0.9924 - val_loss: 0.0755 - val_accuracy: 0.9763
Epoch 3/20
526/526 [==============================] - 15s 28ms/step - loss: 0.0021 - accuracy: 0.9996 - val_loss: 0.0575 - val_accuracy: 0.9744
Epoch 4/20
526/526 [==============================] - 14s 27ms/step - loss: 1.3606e-04 - accuracy: 1.0000 - val_loss: 0.0887 - val_accuracy: 0.9782
Epoch 5/20
526/526 [==============================] - 16s 30ms/step - loss: 3.1304e-05 - accuracy: 1.0000 - val_loss: 0.0407 - val_accuracy: 0.9801
Epoch 6/20
526/526 [==============================] - 14s 27ms/step - loss: 2.5077e-05 - accuracy: 1.0000 - val_loss: 0.1152 - val_accuracy: 0.9517
Epoch 7/20
526/526 [==============================] - 14s 27ms/step - loss: 8.8412e-06 - accuracy: 1.0000 - val_loss: 0.1068

### Saving our model

In [12]:
# Save the model
classifier.save('nutrition1.h5')

### Predicting our results

In [13]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition1.h5") #loading the model for testing

In [14]:
img = image.load_img(r"C:\Users\Varun\Downloads\Nutrition Analysis Using Image Classification-20211029T043804Z-001\Nutrition Analysis Using Image Classification\Flask\Sample_Images\Test_Image2.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)#predicting the classes
pred

array([[0., 1., 0., 0., 0.]], dtype=float32)